In [6]:
from dataclasses import dataclass


@dataclass
class Config:
    pretrain_raw_data = "dataset/tamil_pretrain.txt"
    context_length = 512
    filtered_data = "dataset/filtered_cl_tamil_pretrain.txt"

In [7]:
from datasets import load_dataset

In [16]:
raw_data = load_dataset("text", data_files=Config.filtered_data, streaming=True)

In [18]:
for data in raw_data['train']:
    print(len(data['text']))
    break

1062


In [14]:
import mmap
import re

def clean_whitespaces(text):
    return re.sub(r"\s+", " ", text).strip()


def filter_sentences(file_path, max_len, output_file):
    with open(file_path, "r", encoding="utf-8") as f:

        mmapped_file = mmap.mmap(
            f.fileno(),
            0,
            access=mmap.ACCESS_READ
        )
        with open(output_file, "w") as out_file:
            for line in iter(mmapped_file.readline, b""):
                sentence = line.decode("utf-8").strip()
                sentence = clean_whitespaces(sentence)
                if sentence:
                    if len(sentence) >= max_len:
                        out_file.write(sentence + "\n")
        mmapped_file.close()

In [15]:
filter_sentences(Config.pretrain_raw_data, Config.context_length, "dataset/filtered_cl_tamil_pretrain.txt")